In [ ]:
import warnings
warnings.filterwarnings("ignore")  # because holoviews 1.7 prints a lot of warnings
import kwant
import numpy
import numpy as np
import pandas as pd
from glob import glob
import holoviews as hv
import funcs
hv.notebook_extension()
%opts Image {+axiswise} [colorbar=True aspect='square'] (cmap='viridis')

def add_direction(df):
    xyz = funcs.spherical_coords(1, df['theta'], df['phi'])
    if np.any(np.count_nonzero(xyz, axis=1) > 1):
        raise Exception('Cannot determine direction. Only fields in purley B_x, B_y, or B_z can be used.')
    df['direction'] = np.argmax(xyz, axis=1)
    return df

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/208px-3D_Spherical.svg.png)

# Breaking reflection symmetry with changing the angle of the SC and with different potentials.

_Note that there are not combinations for $g=0$ and `orbital=False`!_

`angle` is the angle of the middle superconductor wrt the $z$-axis.


In [ ]:
df = pd.read_hdf('data/conductance_with_different_potentials_and_SC_angles.hdf')
df = add_direction(df)
params = ['direction', 'V_barrier', 'g', 'alpha', 'orbital', 'mu', 'angle', 'V']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

###  Plot the potential

In [ ]:
def get_cross_section(syst, pos, direction):
    coord = np.array([s.pos for s in syst.sites if s.pos[direction] == pos])
    cross_section = np.delete(coord, direction, 1)
    return cross_section

def plot_potential(angle, V, mu):
    syst_pars = dict(a=10, onsite_disorder=False,
                     L=200, coverage_angle=135, r1=50, r2=70, shape='circle',
                     with_leads=True, with_shell=True, A_correction=True)
    syst_pars['angle'] = angle

    syst = funcs.make_3d_wire(**syst_pars)

    V = eval(V)
    x = 20
    potential = np.array([V(*s.pos) for s in syst.sites if s.pos[0]==x])
    xy = get_cross_section(syst, x, 0)
    return hv.Image(np.rot90(kwant.plotter.mask_interpolate(xy, mu - potential, oversampling=1)[0]),
                   kdims=['$y$', '$z$'], vdims=[r'$\mu$'])

kdims = [hv.Dimension('angle', values=df.angle.unique()), 
         hv.Dimension('$V$', values=df.V.unique()),
         hv.Dimension(r'$\mu$', values=df.mu.unique())]

hv.DynamicMap(plot_potential, kdims=kdims)

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)

# Varying $\mu$ (_has_ $\vec{A}$ correction or no correction for the supercurrent in SC)
_Note that there are not combinations for $g=0$ and `orbital=False`!_

In [ ]:
df = pd.read_hdf('data/conductance_sc_correction.hdf')
df = add_direction(df)
params = ['theta', 'phi', 'V_barrier', 'g', 'alpha', 'orbital', 'mu', 'A_correction']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)

# Varying $\mu$ (has no $\vec{A}$ correction for supercurrent in SC)

In [ ]:
df = pd.read_hdf('data/conductance_vary_mu.hdf')
df['B'], df['theta'], df['phi'] = funcs.cartesian_coords(df.B_x, df.B_y, df.B_z)
params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'theta', 'phi']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)

# First data, $\vec{B}$ in different directions


In [ ]:
df = pd.read_hdf('data/conductance.hdf')
df['B'], df['theta'], df['phi'] = funcs.cartesian_coords(df.B_x, df.B_y, df.B_z)

In [ ]:
params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'theta', 'phi']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)